<a href="https://colab.research.google.com/github/12alain/Datateliers/blob/main/Appartement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importer les packages 
from requests import get
from bs4 import BeautifulSoup as bs

import pandas as pd

In [ ]:
df=pd.DataFrame()
for p in range(1,11):


  url = f'https://sn.coinafrique.com/categorie/appartements?page={p}'
  # Extraire le code html et le stocker dans une variable
  response =get(url)
  html_soup=bs(response.text)
  containers=html_soup.find_all('div',class_='col s6 m4 l3')
  data=[]
  for container in containers : 
    
    try :

      Title=container.find('p',class_='ad__card-description').text.strip()
      Price=container.find('p',class_='ad__card-price').text.replace(' ','').replace('CFA','')
      Adresse=container.find('p',class_='ad__card-location').span.text

      obj = {
          'Title': Title,
          ' Price': int(Price),
          ' Adresse': Adresse
        
      }
      data.append(obj)
      
    except:
      pass
  df1=pd.DataFrame(data)
  df=pd.concat([df1,df],axis=0)


In [ ]:
df.shape

(815, 3)

In [ ]:
df.index=range(df.shape[0])

In [ ]:
df.head()

,Title,Price,Adresse
0,Location appartements 4 pièces - Sicap Liberté,350000,"Sicap Liberté, Dakar, Sénégal"
1,Location Appartement 3 pièces,200000,"Mbao, Dakar, Sénégal"
2,Location appartements 4 pièces - Ouakam,80000,"Ouakam, Dakar, Sénégal"
3,Vente appartements 4 pièces - Yoff virage,195000000,"Yoff, Dakar, Sénégal"
4,Vente villa duplex 5 pièces - Yoff virage,319200000,"Yoff, Dakar, Sénégal"


# Nous allons essayer de nettoyer notre jeu de donnee ici 


In [ ]:
#nous allons spliter le Title en deux variables Titre et Locality  
df[['Titre','Locality']]=df.Title.str.split('-',n=1,expand=True)
#supprimer la variable Title 
df.drop(['Title'],axis=1,inplace=True)

In [ ]:
#nous allons spliter Adresse en Localite , regions et pays
df[['Localite', 'Region','Pays']] = df[' Adresse'].str.split(',', n=2, expand=True)
#supprimer adresse  
df.drop([' Adresse'],axis=1,inplace=True)

In [ ]:
#supprimons la variables pays 
df.drop(['Pays'],axis=1,inplace=True)

In [ ]:
#supprimons la variable locality et gardons localite
df.drop(['Locality'],axis=1,inplace=True)

In [ ]:
#renomons les variables 
df = df.rename(columns={'Titre': 'Title','Localite':'Locality'})

In [ ]:
df

,Price,Title,Locality,Region
0,350000,Location appartements 4 pièces,Sicap Liberté,Dakar
1,200000,Location Appartement 3 pièces,Mbao,Dakar
2,80000,Location appartements 4 pièces,Ouakam,Dakar
3,195000000,Vente appartements 4 pièces,Yoff,Dakar
4,319200000,Vente villa duplex 5 pièces,Yoff,Dakar
...,...,...,...,...
810,350000,Location appartement,Dakar,Sénégal
811,300000,Location appartement 3 pièces,Dakar,Sénégal
812,550000,Location appartement,Mamelles,Dakar
813,75000000,Vente studio,Ngor,Dakar


In [ ]:
#repositionnement des variables 

df = df[['Title', 'Locality', 'Region',' Price']]


In [ ]:
#nous allons essayer de remplacer toute les parties ou il y'a senegal par Dakar  avec ce code qui prend en compte toute les occurences "Sénégal", "Senegal", "SENEGAL", etc

df["Region"] = df["Region"].str.replace("(?i)Sénégal", "Dakar", regex=True)

In [ ]:
df

,Title,Locality,Region,Price
0,Location appartements 4 pièces,Sicap Liberté,Dakar,350000
1,Location Appartement 3 pièces,Mbao,Dakar,200000
2,Location appartements 4 pièces,Ouakam,Dakar,80000
3,Vente appartements 4 pièces,Yoff,Dakar,195000000
4,Vente villa duplex 5 pièces,Yoff,Dakar,319200000
...,...,...,...,...
810,Location appartement,Dakar,Dakar,350000
811,Location appartement 3 pièces,Dakar,Dakar,300000
812,Location appartement,Mamelles,Dakar,550000
813,Vente studio,Ngor,Dakar,75000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815 entries, 0 to 814
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     815 non-null    object
 1   Locality  815 non-null    object
 2   Region    815 non-null    object
 3    Price    815 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 25.6+ KB


In [ ]:
df.duplicated().sum()

110

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df

,Title,Locality,Region,Price
0,Location appartements 4 pièces,Sicap Liberté,Dakar,350000
1,Location Appartement 3 pièces,Mbao,Dakar,200000
2,Location appartements 4 pièces,Ouakam,Dakar,80000
3,Vente appartements 4 pièces,Yoff,Dakar,195000000
4,Vente villa duplex 5 pièces,Yoff,Dakar,319200000
...,...,...,...,...
810,Location appartement,Dakar,Dakar,350000
811,Location appartement 3 pièces,Dakar,Dakar,300000
812,Location appartement,Mamelles,Dakar,550000
813,Vente studio,Ngor,Dakar,75000000


In [ ]:
#importer notre dataset dans un fichier exel 
df.to_excel("data_nettoyee.xlsx", index=False)